# 구글드라이브 연결

In [ ]:
# google drive와 colab연동
from google.colab import auth

auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 함수 호출

In [ ]:
import pandas as pd
import numpy as np
import os
from math import cos,radians
from collections import Counter 
from tqdm import tqdm
from sklearn import ensemble

import matplotlib.pyplot as plt

%matplotlib inline

# 데이터 

In [ ]:
path = "./gdrive/My Drive/dacon/발전량/data/"

train_df = pd.read_csv(path+'./train/train.csv')
sub_df = pd.read_csv(path+'./sample_submission.csv')

test_df = pd.DataFrame()

for i in tqdm(os.listdir(path+'./test/')):
    df = pd.read_csv(path+"./test/"+i)
    df["F_NAME"] = "TEST_"+i
    test_df = pd.concat([test_df,df],axis=0)
    del df

test_df["TY"] = test_df["F_NAME"].apply(lambda x:x[x.find("_")+1:x.find(".")])
test_df["TY"] = test_df["TY"].astype('int')
test_df.sort_values(by=["TY","Day","Hour","Minute"],inplace=True)
test_df = test_df.reset_index(drop=True) 

100%|██████████| 81/81 [00:22<00:00,  3.68it/s]


# FE

In [ ]:
def sun_check(df):
    que = 0
    
    for i in range(len(df)):

        if (df["DHI"][i]>0) & (df["DHI"][i]>0):
            que = 1

        elif (df["DHI"][i] ==0) & (df["DHI"][i]==0):
            que = 0

        df.loc[i,"sun_check"] = que 
        
    df["sun_check"] = df["sun_check"].astype('int')
    
    return df
    
def Altitude_check(cc):
    
    up_unit = round((69-45) / (cc.loc[:cc[cc['Altitude']==69].index[0],'Altitude'].isnull().sum()),2)
    down_unit = round((69-45) / (cc.loc[cc[cc['Altitude']==69].index[0]:,'Altitude'].isnull().sum()),2)

    que = 0

    for i in range(1,len(cc)-1):
        if (cc['Altitude'][i-1] == 0) & np.isnan(cc['Altitude'][i]) & (que == 0):
            cc.loc[i,'Altitude'] = 45
            que = 1

        elif (cc['Altitude'][i-1] >= 45) & np.isnan(cc['Altitude'][i]) & (que == 1):
            cc.loc[i,'Altitude'] = cc['Altitude'][i-1] + up_unit

        elif (cc['Altitude'][i] == 69) & (cc['Altitude'][i] >= 0):
            que = 2 
            continue
            
        elif (cc['Altitude'][i-1] <= 69) & np.isnan(cc['Altitude'][i]) & (que == 2) & (cc['Altitude'][i+1] == 0) :
            cc.loc[i,'Altitude'] = 45
        
        elif (cc['Altitude'][i-1] <= 69) & np.isnan(cc['Altitude'][i]) & (que == 2):
            cc.loc[i,'Altitude'] = cc['Altitude'][i-1] - down_unit
            
    return cc
    

def Altitude(df,train=True):
    
    if train == True:
        df = sun_check(df)
        
        all_df = pd.DataFrame()

        df.loc[(df["Hour"]==12) & (df["Minute"]==30),"Altitude"] = 69
        df.loc[(df["sun_check"]==0),"Altitude"] = 0

        for i in df["Day"].unique():
            cc_fe = Altitude_check(df[df['Day']==i].reset_index(drop=True))
            all_df = pd.concat([all_df,cc_fe],axis=0).reset_index(drop=True)
            
        all_df['Altitude_diff'] = 90 - all_df['Altitude']
        all_df.loc[all_df['Altitude_diff']==90,"Altitude_diff"]=0
        all_df['GHI'] = all_df['DHI'] + (all_df['DNI'] *[cos(radians(i)) for i in all_df['Altitude_diff'].values])

        return all_df

    else:
        df = sun_check(df)
        all_df = pd.DataFrame()

        df.loc[(df["Hour"]==12) & (df["Minute"]==30),"Altitude"] = 69
        df.loc[(df["sun_check"]==0),"Altitude"] = 0
        
        for i in test_df_fe["TY"].unique():
            temp = test_df_fe[test_df_fe["TY"]==i].reset_index(drop=True)

            for j in temp["Day"].unique():
                cc_fe = Altitude_check(temp[temp['Day'] == j].reset_index(drop=True))
                all_df = pd.concat([all_df,cc_fe],axis=0).reset_index(drop=True)
        
        all_df['Altitude_diff'] = 90 - all_df['Altitude']
        all_df.loc[all_df['Altitude_diff']==90,"Altitude_diff"]=0
        all_df['GHI'] = all_df['DHI'] + (all_df['DNI'] *[cos(radians(i)) for i in all_df['Altitude_diff'].values])
        
        return all_df

def time_all(df):
    set_df = df.copy()

    # 변수 만들기 끝
    set_df["ALL_TIME"] = set_df["Hour"]*60 + set_df["Minute"]
    set_df["ALL_TIME"] = set_df["ALL_TIME"].astype('category')

    return set_df

def time_m(df):

    m1 = [1,2,3,4,5,6]
    m2 = [7,8,9,10,11,12]
    m3 = [13,14,15,16,17,18]
    m4 = [19,20,21,22,23,0]

    df.loc[df["Hour"].isin(m1),"M_GROUP"] = 1
    df.loc[df["Hour"].isin(m2),"M_GROUP"] = 2
    df.loc[df["Hour"].isin(m3),"M_GROUP"] = 3
    df.loc[df["Hour"].isin(m4),"M_GROUP"] = 4

    df["M_GROUP"] = df["M_GROUP"].astype('category')

    return df

def dark(df):
    df.loc[df['TARGET']==0,'DARK'] = 1
    df.loc[df['TARGET']!=0,'DARK'] = 0
    
    df['DARK'] = df['DARK'].astype('category')
    
    return df

def time_function(train_df,test_df):

    train_df_fe = time_all(train_df)
    test_df_fe = time_all(test_df)

    train_df_fe = time_m(train_df_fe)
    test_df_fe = time_m(test_df_fe)
    
    train_df_fe = dark(train_df_fe)
    test_df_fe = dark(test_df_fe)
    
    train_df_fe['DARK'] = train_df_fe['DARK'].astype('category') 
    test_df_fe['DARK'] = test_df_fe['DARK'].astype('category') 
    
    return train_df_fe, test_df_fe

In [ ]:
# 새로운 데이터 셋
train_df_fe,test_df_fe = time_function(train_df,test_df)

train_df_fe = Altitude(train_df_fe,train=True)
test_df_fe = Altitude(test_df_fe,train=False)

# slide

In [ ]:
def transform(dataset, target, start_index, end_index, history_size,
                      target_size, step):
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index, 48):
        indices = range(i-history_size, i, step)
        data.append(np.ravel(dataset[indices].T))
        labels.append(target[i:i+target_size])
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

duse_col = ['Day','Hour','Minute','DARK','sun_check','Altitude_diff','Altitude']
x_col = [i for i in train_df_fe if i not in duse_col]
y_col = ['TARGET']

dataset = train_df_fe.loc[:,x_col].values
label = np.ravel(train_df_fe.loc[:,y_col].values)

past_history = 48 * 7
future_target = 48 * 2

### transform train
train_data, train_label = transform(dataset, label, 0,None, past_history,future_target, 1)

In [ ]:
train_data.shape

(1086, 3024)

In [ ]:
test = []

for i in range(81):
    data = []
    tmp = test_df_fe[test_df_fe['TY'] == i].reset_index(drop=True)
    tmp = tmp.loc[:, x_col].values
    tmp = tmp[-past_history:,:]
    data.append(np.ravel(tmp.T))
    data = np.array(data)
    test.append(data)

test = np.concatenate(test, axis=0)

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

for i,(train_index, valid_index) in enumerate(kf.split(train_data)):
  print(i+1,'번째 fold 학습 진행')

  train_x = train_data[train_index]
  train_y = train_label[train_index]

  valid_x = train_data[valid_index]
  valid_y = train_label[valid_index]

  rf = ensemble.RandomForestRegressor(n_estimators=1000, criterion='mae',
                                    max_features = 10, random_state=42,
                                    max_depth = 10,
                                    verbose=False,
                                    n_jobs=-1)

  rf.fit(train_x, train_y)
  print(rf.score(valid_x,valid_y))

  rf_preds = []

  for estimator in rf.estimators_:
      rf_preds.append(estimator.predict(test))
  rf_preds = np.array(rf_preds)

  for i, q in enumerate(np.arange(0.1, 1, 0.1)):
      y_pred = np.percentile(rf_preds, q * 100, axis=0)
      sub_df.iloc[:, i+1] = np.ravel(y_pred)/5    
  print('학습 끝 \n')

1 번째 fold 학습 진행


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.21501261745140354
학습 끝 

2 번째 fold 학습 진행


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.39498510321801866
학습 끝 

3 번째 fold 학습 진행


...

0.08636917702654279
학습 끝 

4 번째 fold 학습 진행


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.4024460952248175
학습 끝 

5 번째 fold 학습 진행


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.3070902877577591
학습 끝 



In [ ]:
sub_df = pd.read_csv(path+'./sample_submission.csv')

for i, q in enumerate(np.arange(0.1, 1, 0.1)):
    y_pred = np.percentile(rf_preds, q * 100, axis=0)
    sub_df.iloc[:, i+1] = np.ravel(y_pred)    

In [ ]:
sub_df.to_csv(path+'ranfo_q_ver1.csv',index=False)